<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE8-WEEK2/m08w02_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aspect Based sentiment analysis

In [1]:
# install lib
! pip install -q datasets==3.2.0

from datasets import load_dataset

ds = load_dataset("thainq107/abte-restaurants")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3602 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [2]:
#tokenization
from transformers import AutoTokenizer

tokenizer =  AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

# định nghĩa mộ hàm token và sắp xếp labels
def tokenize_and_align_labels(examples):
  tokenized_inputs = []
  labels = []
  for tokens, tags in zip(examples["Tokens"], examples["Tags"]):
      #tokens = tokens.replace("'", "").strip("][").split(',')
      #tags = tags.strip("][").split(',')

      bert_tokens = []
      bert_tags = []
      for i in range(len(tokens)):
        t = tokenizer.tokenize(tokens[i])
        bert_tokens += t
        bert_tags += [int(tags[i])]*len(t)

      bert_ids = tokenizer.convert_tokens_to_ids(bert_tokens)
      tokenized_inputs.append(bert_ids)
      labels.append(bert_tags)

  return {
      'input_ids': tokenized_inputs,
      'labels': labels
}



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Đoạn code này tạo một Data Collator giúp xử lý và sắp xếp dữ liệu đầu vào một cách tự động khi huấn luyện mô hình BERT (hoặc các mô hình tương tự) trong bài toán gán nhãn từ (Token Classification), chẳng hạn như NER (Named Entity Recognition)

In [3]:
preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [4]:
# data collector
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [8]:
!pip install -q seqeval==1.2.2
import numpy as np
from seqeval.metrics import accuracy_score

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax( predictions, axis=2)
  true_predictions = [
      [str(p) for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [str(l) for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  results = accuracy_score(true_labels, true_predictions)
  return {"accuracy": results}

In [6]:
from transformers import AutoModelForTokenClassification

id2label = {
    0: 'O',
    1: 'B-aspect',
    2: 'I-aspect'
}
label2id = {
    'O': 0,
    'B-aspect': 1,
    'I-aspect': 2
}

model = AutoModelForTokenClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import os
os.environ['WANDB_DISABLED'] = 'true'
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="abte-restaurants-distilbert-base-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.221901,0.937867
2,No log,0.234440,0.940234
3,0.047400,0.229147,0.942877
4,0.047400,0.256447,0.939800
5,0.019100,0.262385,0.942917


TrainOutput(global_step=1130, training_loss=0.030923861950899648, metrics={'train_runtime': 137.5139, 'train_samples_per_second': 130.969, 'train_steps_per_second': 8.217, 'total_flos': 241258883931900.0, 'train_loss': 0.030923861950899648, 'epoch': 5.0})

In [12]:
from transformers import pipeline

token_classifier = pipeline(
    model = "thainq107/abte-restaurants-distilbert-base-uncased",
    aggregation_strategy ="simple"
)

test_sentence = 'The bread is top norch as well'
results = token_classifier(test_sentence)
results

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity_group': 'Term',
  'score': 0.87228775,
  'word': 'bread',
  'start': 4,
  'end': 9},
 {'entity_group': 'Term',
  'score': 0.39604083,
  'word': 'nor',
  'start': 17,
  'end': 20}]